In [1]:
!git clone https://github.com/gagroo-jeevansh/bertensm.git

Cloning into 'bertensm'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 82 (delta 41), reused 53 (delta 16), pack-reused 0
Receiving objects: 100% (82/82), 173.99 KiB | 4.05 MiB/s, done.
Resolving deltas: 100% (41/41), done.
Filtering content: 100% (6/6), 1.40 GiB | 58.74 MiB/s, done.


In [2]:
%%bash
pip install numpy torch datasets transformers~=4.28.0 evaluate tqdm --quiet
pip freeze | grep -E '^numpy|^torch|^datasets|^transformers|^evaluate'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 61.4 MB/s eta 0:00:00
datasets==2.19.0
evaluate==0.4.2
numpy==1.25.2
torch @ https://download.pytorch.org/whl/cu121/torch-2.2.1%2Bcu121-cp310-cp310-linux_x86_64.whl#sha256=1adf430f01ff649c848ac021785e18007b0714fdde68e4e65bd0c640bf3fb8e1
torchaudio @ https://download.pytorch.org/whl/cu121/torchaudio-2.2.1%2Bcu121-cp310-cp310-linux_x86_64.whl#sha256=23f6236429e2bf676b8

In [3]:
from datasets import load_dataset
import numpy as np
import torch

# set seed for reproducibility
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
# clone dataset
# source: https://huggingface.co/datasets/ucberkeley-dlab/measuring-hate-speech
raw_datasets = load_dataset("ucberkeley-dlab/measuring-hate-speech")

print(f"Number of Columns: {raw_datasets['train'].num_columns}")
print(f"Number of Rows: {raw_datasets['train'].num_rows}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/135556 [00:00<?, ? examples/s]

Number of Columns: 131
Number of Rows: 135556


In [4]:
# keep only text and specific targets
raw_columns = raw_datasets['train'].column_names
keep_columns = ['text', 'target_race', 'target_religion', 'target_origin', 'target_gender', 'target_sexuality', 'target_age', 'target_disability']
remove_columns = set(raw_columns)-set(keep_columns)

preprocessed_datasets = raw_datasets.remove_columns(remove_columns)
preprocessed_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'target_race', 'target_religion', 'target_origin', 'target_gender', 'target_sexuality', 'target_age', 'target_disability'],
        num_rows: 135556
    })
})

In [5]:
column_mapping = {column:column.split('_')[1] for column in keep_columns if column.startswith('target')}
print(f"COLUMN_MAPPING: {column_mapping}")

preprocessed_datasets = preprocessed_datasets.rename_columns(column_mapping)
preprocessed_datasets

COLUMN_MAPPING: {'target_race': 'race', 'target_religion': 'religion', 'target_origin': 'origin', 'target_gender': 'gender', 'target_sexuality': 'sexuality', 'target_age': 'age', 'target_disability': 'disability'}


DatasetDict({
    train: Dataset({
        features: ['text', 'race', 'religion', 'origin', 'gender', 'sexuality', 'age', 'disability'],
        num_rows: 135556
    })
})

In [6]:
# get two-way label and label id
ID2LABEL = {}
LABEL2ID = {}

label_id = 0
for label in preprocessed_datasets['train'].features.keys():
    if label in ['text']:
        continue

    ID2LABEL[label_id] = label
    LABEL2ID[label] = label_id

    label_id += 1

print(f"ID2LABEL:\n{ID2LABEL}\n")
print(f"LABEL2ID:\n{LABEL2ID}")

ID2LABEL:
{0: 'race', 1: 'religion', 2: 'origin', 3: 'gender', 4: 'sexuality', 5: 'age', 6: 'disability'}

LABEL2ID:
{'race': 0, 'religion': 1, 'origin': 2, 'gender': 3, 'sexuality': 4, 'age': 5, 'disability': 6}


In [7]:
# get target label counts and percentages
label_counts = {}
label_percentages = {}

for label in LABEL2ID:
    label_counts[label] = sum(preprocessed_datasets['train'][label])
    label_percentages[label] = float(f"{sum(preprocessed_datasets['train'][label]) / len(preprocessed_datasets['train'])*100:.2f}")

print(f"LABEL_COUNTS:\n{label_counts}\n")
print(f"LABEL_PERCENTAGES:\n{label_percentages}")

LABEL_COUNTS:
{'race': 48352, 'religion': 26182, 'origin': 24372, 'gender': 40439, 'sexuality': 22407, 'age': 2005, 'disability': 3716}

LABEL_PERCENTAGES:
{'race': 35.67, 'religion': 19.31, 'origin': 17.98, 'gender': 29.83, 'sexuality': 16.53, 'age': 1.48, 'disability': 2.74}


In [8]:
def create_labels(batch):
    # one-hot encode targets for training
    batch['labels'] = [[float(batch[label][i]) for label in LABEL2ID] for i in range(len(batch['text']))]
    return batch

preprocessed_datasets = preprocessed_datasets.map(create_labels, batched=True, remove_columns=LABEL2ID.keys())
preprocessed_datasets

Map:   0%|          | 0/135556 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 135556
    })
})

In [9]:
from datasets import DatasetDict

# train (80%), validation (10%), test (10%) split
train_test_datasets = preprocessed_datasets['train'].train_test_split(test_size=0.2, seed=SEED, shuffle=True)
validation_test_datasets = train_test_datasets['test'].train_test_split(test_size=0.5, seed=SEED, shuffle=True)

preprocessed_datasets = DatasetDict({
    'train': train_test_datasets['train'],
    'validation': validation_test_datasets['train'],
    'test': validation_test_datasets['test']
})
preprocessed_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 108444
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 13556
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 13556
    })
})

In [10]:
from transformers import AutoTokenizer

paths = {
    'bert': './BERT',
    'roberta': './RoBERTa',
    'distilbert': './Distilbert',
    'electra': './ELECTRA',
    'albert': './ALBERT',
}

dict = {
    'bert': {'model': None, 'pipeline': None, 'tokenizer': AutoTokenizer.from_pretrained('bert-base-uncased')},
    'roberta': {'model': None, 'pipeline': None, 'tokenizer': AutoTokenizer.from_pretrained('FacebookAI/roberta-base')},
    'distilbert': {'model': None, 'pipeline': None, 'tokenizer': AutoTokenizer.from_pretrained('distilbert-base-uncased')},
    'electra': {'model': None, 'pipeline': None, 'tokenizer': AutoTokenizer.from_pretrained('google/electra-base-generator')},
    'albert': {'model': None, 'pipeline': None, 'tokenizer': AutoTokenizer.from_pretrained('albert-base-v2')}
}

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

# **Ensemble Pipeline Dictionary**

In [11]:
from transformers import AutoModelForSequenceClassification
from transformers import pipeline

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for key in paths:
    dict[key]['model'] = AutoModelForSequenceClassification.from_pretrained(paths[key], num_labels=len(LABEL2ID))
    dict[key]['model'].to(device)
    dict[key]['pipeline'] = pipeline(
        task = 'text-classification',
        model = dict[key]['model'],
        tokenizer = dict[key]['tokenizer'],
        device = torch.device('cpu'),
        top_k = None
    )

# **Addition of Binary Classifier**

In [12]:
tok = AutoTokenizer.from_pretrained("facebook/roberta-hate-speech-dynabench-r4-target")
tmp = AutoModelForSequenceClassification.from_pretrained("facebook/roberta-hate-speech-dynabench-r4-target")
binary_pipe = pipeline("text-classification", model=tmp, tokenizer=tok)

tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/816 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [13]:
from collections import defaultdict

def predict(race_hate_text):

  bin = False
  binscore = 1-binary_pipe(race_hate_text)[0]['score']
  if(binary_pipe(race_hate_text)[0]['label']!='hate'):
    print("The text is not hate speech!")
    bin = True
  else:
    print("Hate Speech Detected")

  # Initialize a defaultdict to store label scores
  ensemble_scores = defaultdict(float)

  # Number of models for averaging
  num_models = len(paths)

  # Get predictions from each model
  for key in dict:
      pipeline_output = dict[key]['pipeline'](race_hate_text)
      for prediction in pipeline_output[0]:
          label = prediction['label']
          score = prediction['score']
          ensemble_scores[label] += score / num_models

  # Convert ensemble_scores to a list of dictionaries for easy sorting and printing
  ensemble_predictions = [{'label': label, 'score': score*binscore if bin==True else score} for label, score in ensemble_scores.items()]

  # Sort predictions by score in descending order
  sorted_ensemble_predictions = sorted(ensemble_predictions, key=lambda x: x['score'], reverse=True)

  return sorted_ensemble_predictions

predict("God is great")

The text is not hate speech!


[{'label': 'religion', 'score': 0.00013085632887026576},
 {'label': 'gender', 'score': 3.9167670518924775e-06},
 {'label': 'race', 'score': 2.9929596673028413e-06},
 {'label': 'sexuality', 'score': 2.70283660672721e-06},
 {'label': 'origin', 'score': 2.2537066223238256e-06},
 {'label': 'disability', 'score': 9.721726053180113e-07},
 {'label': 'age', 'score': 9.694522299430465e-07}]

In [14]:
predict("Latinas are a despicable race")

Hate Speech Detected


[{'label': 'race', 'score': 0.9669194936752321},
 {'label': 'origin', 'score': 0.12247454524040222},
 {'label': 'gender', 'score': 0.0176072932779789},
 {'label': 'religion', 'score': 0.00730249141342938},
 {'label': 'sexuality', 'score': 0.004138418170623481},
 {'label': 'age', 'score': 0.002757122949697078},
 {'label': 'disability', 'score': 0.0026104330318048596}]